# 06 — MNIST Handwritten Digit Classification

**Goal**: Train a CNN in PyTorch on MNIST, quantize to i8, run inference in NANO-RUST.

Architecture: `Conv2D(1→8, 3×3) → ReLU → MaxPool(2×2) → Conv2D(8→16, 3×3) → ReLU → MaxPool(2×2) → Flatten → Dense(400→10)`

### Setup
1. Ensure you're using the **NanoRust (venv)** kernel
2. First cell builds the library via `maturin develop`

In [1]:
%run _setup.py
setup_all()

⚠️  WARNING: Not running inside a virtual environment!
   Create one: python -m venv .venv
   Activate:   .venv\Scripts\activate  (Windows)
   Install:    pip install maturin numpy torch torchvision jupyter ipykernel
   Register:   python -m ipykernel install --user --name nanorust
🔨 Building nano_rust_py (target: C:\Users\suoya\.nanorust_target)...
❌ Build failed:
ðŸ’¥ maturin failed
  Caused by: Failed to parse Cargo.toml at c:\Users\suoya\OneDrive\Documents\nanorust\Cargo.toml
  Caused by: TOML parse error at line 1, column 1
  |
1 | [workspace]
  | ^
missing field `package`




RuntimeError: maturin develop failed

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from nano_rust_utils import quantize_to_i8, quantize_weights, calibrate_model
import nano_rust_py
print('✅ All modules loaded')

## Step 1: Load MNIST Dataset

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

print(f'Train samples: {len(train_dataset)}')
print(f'Test samples:  {len(test_dataset)}')
print(f'Input shape:   {train_dataset[0][0].shape}')

## Step 2: Define & Train CNN

In [ ]:
class MNISTNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1),   # [1,28,28]→[8,28,28]
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                                     # →[8,14,14]
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),  # →[16,14,14]
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                                     # →[16,7,7]
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),                                           # →[784]
            nn.Linear(16 * 7 * 7, 10),
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = MNISTNet()
print(model)
print(f'Parameters: {sum(p.numel() for p in model.parameters()):,}')

In [ ]:
# Training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

EPOCHS = 3  # 3 epochs is enough for >97% accuracy
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        total += target.size(0)
    
    acc = 100. * correct / total
    print(f'Epoch {epoch+1}/{EPOCHS} — Loss: {total_loss/len(train_loader):.4f}, Acc: {acc:.1f}%')

model = model.cpu().eval()
print('✅ Training complete')

## Step 3: PyTorch Test Accuracy

In [ ]:
model.eval()
correct_pt = 0
total_pt = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        pred = output.argmax(dim=1)
        correct_pt += pred.eq(target).sum().item()
        total_pt += target.size(0)
pt_accuracy = 100. * correct_pt / total_pt
print(f'PyTorch Test Accuracy: {pt_accuracy:.2f}% ({correct_pt}/{total_pt})')


## Step 4: Quantize & Build NANO-RUST Model

We need to export the model as a flat `nn.Sequential` for `calibrate_model`.

In [ ]:
# Flatten the model into nn.Sequential for calibration
flat_
# GPU Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
if device.type == 'cuda':
    print(f'GPU: {torch.cuda.get_device_name(0)}')

model = nn.Sequential(
    model.features[0],  # Conv2d(1,8,3,1,1)
    model.features[1],  # ReLU
    model.features[2],  # MaxPool2d(2,2)
    model.features[3],  # Conv2d(8,16,3,1,1)
    model.features[4],  # ReLU
    model.features[5],  # MaxPool2d(2,2)
    nn.Flatten(),        # Flatten
    model.classifier[1], # Linear(784,10)
)
flat_model.eval()
print('Flat model:', flat_model)

q_weights = quantize_weights(flat_model)
print(f"\nQuantized layers: {list(q_weights.keys())}")
for name, info in q_weights.items():
    if info['weights'] is not None:
        print(f"  {name} ({info['type']}): weights={info['weights'].shape}, scale={info['weight_scale']:.6f}")

model = model.to(device)
print("Model moved to device")


In [ ]:
# Calibrate with a representative sample
cal_image = test_dataset[0][0].unsqueeze(0)  # [1, 1, 28, 28]
q_cal, cal_scale = quantize_to_i8(cal_image.numpy().flatten())

requant = calibrate_model(flat_model, cal_image, q_weights, cal_scale)
print('Calibration results:')
for name, params in requant.items():
    if isinstance(params[0], str):
        print(f'  Layer {name}: {params[0]} scale_mult={params[1]}, scale_shift={params[2]}')
    else:
        print(f'  Layer {name}: M={params[0]}, shift={params[1]}')


In [ ]:
# Build NANO-RUST model
def build_nano_model(q_weights, requant):
    nano = nano_rust_py.PySequentialModel(input_shape=[1, 28, 28], arena_size=131072)
    
    # Conv2d(1→8, 3×3, stride=1, pad=1)
    m, s, bc = requant['0']
    nano.add_conv2d_with_requant(
        q_weights['0']['weights'].flatten().tolist(), bc,
        1, 8, 3, 3, 1, 1, m, s)
    nano.add_relu()
    nano.add_max_pool2d(2, 2, 2)
    
    # Conv2d(8→16, 3×3, stride=1, pad=1)
    m, s, bc = requant['3']
    nano.add_conv2d_with_requant(
        q_weights['3']['weights'].flatten().tolist(), bc,
        8, 16, 3, 3, 1, 1, m, s)
    nano.add_relu()
    nano.add_max_pool2d(2, 2, 2)
    
    # Flatten + Dense(784→10)
    nano.add_flatten()
    m, s, bc = requant['7']
    nano.add_dense_with_requant(
        q_weights['7']['weights'].flatten().tolist(), bc, m, s)
    
    return nano

nano = build_nano_model(q_weights, requant)
print('✅ NANO-RUST model built')


## Step 5: NANO-RUST Test Accuracy

In [ ]:
# Test on the full test set
correct_nano = 0
correct_match = 0  # Agreement between PyTorch and NANO-RUST
total_test = 0
max_diffs = []

N_TEST = min(1000, len(test_dataset))  # Test on first 1000 samples

for i in range(N_TEST):
    image, label = test_dataset[i]
    image_np = image.numpy().flatten()
    q_image, _ = quantize_to_i8(image_np)
    
    # NANO-RUST inference
    nano_model = build_nano_model(q_weights, requant)
    nano_out = nano_model.forward(q_image.tolist())
    nano_class = int(np.argmax(nano_out))
    
    # PyTorch reference
    with torch.no_grad():
        pt_out = flat_model(image.unsqueeze(0)).numpy().flatten()
    pt_class = int(np.argmax(pt_out))
    
    # Compare
    q_pt, _ = quantize_to_i8(pt_out)
    nano_arr = np.array(nano_out, dtype=np.int8)
    diff = np.abs(q_pt.astype(np.int32) - nano_arr.astype(np.int32))
    max_diffs.append(int(np.max(diff)))
    
    if nano_class == label:
        correct_nano += 1
    if nano_class == pt_class:
        correct_match += 1
    total_test += 1

nano_accuracy = 100. * correct_nano / total_test
agreement = 100. * correct_match / total_test

print('=' * 60)
print('       MNIST CLASSIFICATION RESULTS')
print('=' * 60)
print(f'PyTorch Accuracy:      {pt_accuracy:.2f}%')
print(f'NANO-RUST Accuracy:    {nano_accuracy:.2f}% (on {N_TEST} samples)')
print(f'Classification Match:  {agreement:.1f}%')
print(f'Max i8 Diff (median):  {int(np.median(max_diffs))}')
print(f'Max i8 Diff (95th):    {int(np.percentile(max_diffs, 95))}')
print(f'Max i8 Diff (max):     {max(max_diffs)}')
print('=' * 60)
print(f'{"✅ PASS" if agreement > 90 else "❌ FAIL"}: {agreement:.1f}% classification agreement')


## Step 6: Memory Budget

In [ ]:
print('\n📊 Memory Budget for MCU Deployment')
print('=' * 50)
total_flash = 0
for name, info in q_weights.items():
    if info['weights'] is not None:
        w_bytes = info['weights'].nbytes
        b_bytes = len(requant[name][2]) if name in requant and not isinstance(requant[name][0], str) else 0
        total_flash += w_bytes + b_bytes
        print(f'  Layer {name} ({info["type"]}): {w_bytes + b_bytes:>6} bytes')

# Arena = largest intermediate buffer
# After Conv1: 8*28*28 = 6272 bytes
# After Pool1: 8*14*14 = 1568 bytes
# After Conv2: 16*14*14 = 3136 bytes
# After Pool2: 16*7*7 = 784 bytes
# im2col buffer: max(1*3*3*28*28, 8*3*3*14*14) = max(7056, 14112) = 14112
arena_est = 32768  # Conservative estimate

print(f'\n  Total Flash: {total_flash:>6} bytes ({total_flash/1024:.1f} KB)')
print(f'  Arena (RAM): {arena_est:>6} bytes ({arena_est/1024:.1f} KB)')
print(f'  Total:       {total_flash+arena_est:>6} bytes ({(total_flash+arena_est)/1024:.1f} KB)')
print(f'\n  Fits ESP32 (520KB RAM)?     {"✅" if arena_est < 520*1024 else "❌"}')
print(f'  Fits STM32F4 (192KB RAM)?   {"✅" if arena_est < 192*1024 else "❌"}')
print(f'  Fits STM32L4 (96KB RAM)?    {"✅" if arena_est < 96*1024 else "❌"}')
